In [ ]:
import json
from matplotlib import pyplot as plt
import xtrack as xt
import numpy as np
import plotly.graph_objects as go


In [ ]:
BEAM  = "b1"

In [ ]:
collider = xt.Multiline.from_json('../collider/collider.json')
line = collider[f'lhc{BEAM}']

Create the knobs for wire current and wire distance to the beam

In [ ]:
# IP 1
line.vars[f'i_wire_ip1.{BEAM}'] = 0.0 
line.vars[f'd_wire_ip1.{BEAM}'] = 0.01 

# IP 5
line.vars[f'i_wire_ip5.{BEAM}'] = 0.0
line.vars[f'd_wire_ip5.{BEAM}'] = 0.01

Insert (unconfigured) wires on the line

In [ ]:

side = 'r' if BEAM == 'b2' else 'l'
sign = 1 if BEAM == 'b2' else -1
l_name_wire = [f'bbwc.t.4{side}1', f'bbwc.b.4{side}1', f'bbwc.e.4{side}5', f'bbwc.i.4{side}5']
l_name_tct = [f'tctpv.4{side}1', f'tctpv.4{side}1', f'tctph.4{side}5', f'tctph.4{side}5']
l_h_dist = sign * np.array([0., 0., 1., -1.])
l_v_dist = sign * np.array([1., -1., 0., 0.])

# Tw to get the position of the tct, but need to discard tracker afterwards to unfreeze the line
tw = line.twiss()
l_s_tct = [((tw.rows[f'{name_tct}.{BEAM}_entry'].s + tw.rows[f'{name_tct}.{BEAM}_exit'].s)/2)[0] for name_tct in l_name_tct]
line.discard_tracker()
for name_wire, name_tct, h_dist, v_dist, s_tct in zip(l_name_wire, l_name_tct, l_h_dist, l_v_dist, l_s_tct):
    line.insert_element(name=f'{name_wire}.{BEAM}',
                        element=xt.Wire(
                            L_phy=1, 
                            L_int=2,
                            current= 0.0,
                            xma=h_dist, 
                            yma= v_dist # very far from the beam
                            ),
                        at_s = s_tct
                        )


Get closed orbit position at the location of the wire

In [ ]:
tw = line.twiss()
# Careful, tct are repeated so only take one out of two
x_tct_ip1, x_tct_ip5 = [((tw.rows[f'{name_tct}.{BEAM}_entry'].x + tw.rows[f'{name_tct}.{BEAM}_exit'].x)/2)[0] for name_tct in l_name_tct[::2]]
y_tct_ip1, y_tct_ip5 = [((tw.rows[f'{name_tct}.{BEAM}_entry'].y + tw.rows[f'{name_tct}.{BEAM}_exit'].y)/2)[0] for name_tct in l_name_tct[::2]]


Create corresponding knob for closed orbit

In [ ]:
for co_wire, co in zip([f'co_y_wire_ip1.{BEAM}', f'co_x_wire_ip1.{BEAM}', f'co_y_wire_ip5.{BEAM}', f'co_x_wire_ip5.{BEAM}'], [y_tct_ip1, x_tct_ip1, y_tct_ip5, x_tct_ip5]):
    line.vars[co_wire] = co


In [ ]:
l_name_wire

Create knob for current scaling, and wire distance scaling

In [ ]:
for name_wire in l_name_wire:
    
    # Check IP
    if 'r1' in name_wire or 'l1' in name_wire:
        ip = 1
    elif 'r5' in name_wire or 'l5' in name_wire:
        ip = 5
    else:
        raise ValueError('Invalid wire name')
    
    # Check plane
    if '.t.' in name_wire or '.b.' in name_wire:
        plane = 'y'
        if '.t.' in name_wire:
            sign = 1
        else:
            sign = -1
    elif '.e.' in name_wire or '.i.' in name_wire:
        plane = 'x'
        if '.e.' in name_wire:
            sign = 1
        else:
            sign = -1
    else:
        raise ValueError('Invalid wire name')
    
    # Assign knob
    line.element_refs[f'{name_wire}.{BEAM}'].current = line.vars[f'i_wire_ip{ip}.{BEAM}']
    if plane == 'y':
        line.element_refs[f'{name_wire}.{BEAM}'].yma = line.vars[f'd_wire_ip{ip}.{BEAM}'] + line.vars[f'co_y_wire_ip{ip}.{BEAM}']
    else:
        line.element_refs[f'{name_wire}.{BEAM}'].xma = sign*line.vars[f'd_wire_ip{ip}.{BEAM}'] + line.vars[f'co_x_wire_ip{ip}.{BEAM}']


Close separation and crossing in IP2 and IP8 and ensure that orbit is flat

In [ ]:
for ip in [2,8]:
        collider.vars[f'on_x{ip}h'] = 0.0
        collider.vars[f'on_x{ip}v'] = 0.0
        collider.vars[f'on_sep{ip}h'] = 0.0
        collider.vars[f'on_sep{ip}v'] = 0.0

for ip in [1,2,5,8]:
        print(8*'*', f'IP{ip}', 8*'*')
        if ip in [2,8]:
                print(f'on_sep{ip}h:\t ', collider.vars[f'on_sep{ip}h']._get_value())
                print(f'on_sep{ip}v:\t ', collider.vars[f'on_sep{ip}v']._get_value())
        else:
                print(f'on_x{ip}:\t\t ', collider.vars[f'on_x{ip}']._get_value())
                print(f'on_sep{ip}:\t ', collider.vars[f'on_sep{ip}']._get_value())
        print(f'on_oh{ip}:\t\t ', collider.vars[f'on_oh{ip}']._get_value())
        print(f'on_ov{ip}:\t\t ', collider.vars[f'on_ov{ip}']._get_value())
        print(f'on_a{ip}:\t\t ', collider.vars[f'on_a{ip}']._get_value())

print(8*'*', 'others settings', 8*'*')
print('on_alice_normalized:\t', collider.vars['on_alice_normalized']._get_value())
print('on_lhcb_normalized:\t', collider.vars['on_lhcb_normalized']._get_value())
print('on_disp:\t\t', collider.vars['on_disp']._get_value())

Load all knobs

In [ ]:
#Lod knob for both IPs
with open(f'/afs/cern.ch/work/c/cdroin/private/example_DA_study_runIII_wire/master_study/master_jobs/knobs_wire/knob_dict_350A_8sigma@30cm_ip1_beta30_{BEAM}.json') as f:
    data_ip1 = json.load(f)
    
with open(f'/afs/cern.ch/work/c/cdroin/private/example_DA_study_runIII_wire/master_study/master_jobs/knobs_wire/knob_dict_350A_8sigma@30cm_ip5_beta30_{BEAM}.json') as f:
    data_ip5 = json.load(f)


Update wire distances

In [ ]:
side = 'r' if BEAM == 'b2' else 'l'
line.vars[f'd_wire_ip1.{BEAM}'] = data_ip1['tct_opening_in_sigma'] * data_ip1[f'sigma_y_at_tctpv_4{side}1_{BEAM}'] + data_ip1['wire_retraction']
line.vars[f'd_wire_ip5.{BEAM}'] = data_ip5['tct_opening_in_sigma'] * data_ip5[f'sigma_x_at_tctph_4{side}5_{BEAM}'] + data_ip5['wire_retraction']

Assert initial k are correct in the knob

In [ ]:
for k0 in data_ip1['k_0']:
    assert data_ip1['k_0'][k0] == line.vars[k0]._get_value()

Define list of k for the matching (setting them to 0 initially)

In [ ]:
k_list_for_matching = [
    f"kq5.l1{BEAM}",
    f"kq5.r1{BEAM}",
    f"kq6.l1{BEAM}",
    f"kq6.r1{BEAM}",
    f"kq7.l1{BEAM}",
    f"kq7.r1{BEAM}",
    f"kq8.l1{BEAM}",
    f"kq8.r1{BEAM}",
    f"kq9.l1{BEAM}",
    f"kq9.r1{BEAM}",
    f"kq10.l1{BEAM}",
    f"kq10.r1{BEAM}",
    f"kqtl11.r1{BEAM}",
    f"kqt12.r1{BEAM}",
    f"kqt13.r1{BEAM}",
    f"kq4.l5{BEAM}",
    f"kq4.r5{BEAM}",
    f"kq5.l5{BEAM}",
    f"kq5.r5{BEAM}",
    f"kq6.l5{BEAM}",
    f"kq6.r5{BEAM}",
    f"kq7.l5{BEAM}",
    f"kq7.r5{BEAM}",
    f"kq8.l5{BEAM}",
    f"kq8.r5{BEAM}",
    f"kq9.l5{BEAM}",
    f"kq9.r5{BEAM}",
    f"kq10.l5{BEAM}",
    f"kq10.r5{BEAM}",
    f"kqtl11.r5{BEAM}",
    f"kqt12.r5{BEAM}",
    f"kqt13.r5{BEAM}",
]


Define/reset the delta_k, used to scale the knob with current

In [ ]:
def reset_delta_k(k_list):
    for kk in k_list:
        collider.vars[f"{kk}_delta"] = 0.000000
reset_delta_k(k_list_for_matching)

Set the delta_k

In [ ]:
for data in [data_ip1, data_ip5]:
    for delta_k in data['k_delta']:
        collider.vars[f'{delta_k}_delta'] = data['k_delta'][delta_k]

Set the k

In [ ]:
for k in k_list_for_matching:
        collider.vars[f'{k}_0'] = collider.vars[k]._get_value()
        # collider.vars[f'{k}_delta'] = 0.000000
        if 'r1' in k or 'l1' in k:
                collider.vars[k] = collider.vars[f'{k}_0'] + collider.vars[f'{k}_delta']*collider.vars[f'i_wire_ip1.{BEAM}']/350
        elif 'r5' in k or 'l5' in k:
                collider.vars[k] = collider.vars[f'{k}_0'] + collider.vars[f'{k}_delta']*collider.vars[f'i_wire_ip5.{BEAM}']/350

Set the current

In [ ]:
def plot_beta():
    fig = go.Figure()

    # Get ref current
    line.vars[f'i_wire_ip1.{BEAM}'] = data_ip1[f'i_wire_ip1.{BEAM}']
    line.vars[f'i_wire_ip5.{BEAM}'] = data_ip5[f'i_wire_ip5.{BEAM}']
    tw_ref = line.twiss()

    # Add traces, one for each slider step
    l_steps = np.arange(350, -50, -50)
    for idx, step in enumerate(l_steps):
        line.vars[f'i_wire_ip1.{BEAM}'] = step
        line.vars[f'i_wire_ip5.{BEAM}'] = step
        tw = line.twiss()
        fig.add_trace(
            go.Scattergl(
                visible=False,
                line=dict(color="#00CED1", width=6),
                name="Betx",
                x=tw['s'][::20],
                y=abs(tw['betx'][::20] - tw_ref['betx'][::20]) / tw_ref['betx'][::20],
            )
        )
        fig.add_trace(
            go.Scattergl(
                visible=False,
                line=dict(color="orange", width=6),
                name="Bety",
                x=tw['s'][::20],
                y=abs(tw['bety'][::20] - tw_ref['bety'][::20]) / tw_ref['bety'][::20],
            )
        )

    # Make 0th trace visible
    fig.data[0].visible = True
    fig.data[1].visible = True


    # Create and add slider
    steps = []
    for i in range(0,len(fig.data),2):
        step = dict(
            method="update",
            args=[{"visible": [False] * len(fig.data)},],  # layout attribute
        label=str(l_steps[i//2]))
        step["args"][0]["visible"][i] = True
        step["args"][0]["visible"][i+1] = True
        steps.append(step)

    sliders = [dict(
        active=0,
        currentvalue={"prefix": "Wire current (A): "},
        pad={"t": 50},
        steps=steps,
    )]

    fig.update_layout(
        sliders=sliders,
        yaxis=dict(range=[-0.1, 0.1], autorange=False)
    )
    fig.update_layout(template = 'plotly_white')
    # Label y axis
    fig.update_yaxes(title_text=r'Relative error on beta functions')
    return fig



In [ ]:
fig = plot_beta()
fig.show()

In [ ]:
def plot_orbit():
    fig = go.Figure()

    # Get ref current
    line.vars[f'i_wire_ip1.{BEAM}'] = data_ip1[f'i_wire_ip1.{BEAM}']
    line.vars[f'i_wire_ip5.{BEAM}'] = data_ip5[f'i_wire_ip5.{BEAM}']
    tw_ref = line.twiss()

    # Add traces, one for each slider step
    l_steps = np.arange(350, -50, -50)
    for step in l_steps:
        line.vars[f'i_wire_ip1.{BEAM}'] = step
        line.vars[f'i_wire_ip5.{BEAM}'] = step
        tw = line.twiss()
        fig.add_trace(
            go.Scattergl(
                visible=False,
                line=dict(color="#00CED1", width=6),
                name="x",
                x=tw['s'][::20],
                y=np.log(abs(tw['x'][::20]- tw_ref['x'][::20])+1e-26)
                )
            )
        fig.add_trace(
            go.Scattergl(
                visible=False,
                line=dict(color="orange", width=6),
                name="y",
                x=tw['s'][::20],
                y=np.log(abs(tw['y'][::20]- tw_ref['y'][::20])+1e-26)
                )
            )
        
    # Make 0th trace visible
    fig.data[0].visible = True
    fig.data[1].visible = True

    # Create and add slider
    steps = []
    for i in range(0,len(fig.data),2):
        step = dict(
            method="update",
            args=[{"visible": [False] * len(fig.data)},],  # layout attribute
        label=str(l_steps[i//2]))
        step["args"][0]["visible"][i] = True
        step["args"][0]["visible"][i+1] = True
        steps.append(step)

    sliders = [dict(
        active=0,
        currentvalue={"prefix": "Wire current (A): "},
        pad={"t": 50},
        steps=steps,
    )]

    fig.update_layout(
        sliders=sliders,
        yaxis=dict(range=[-28, -8], autorange=False)
    )
    fig.update_layout(template = 'plotly_white')

    # Label y axis
    fig.update_yaxes(title_text=r'Log(Absolute error on closed-orbit)')
    
    return fig




In [ ]:
fig = plot_orbit()
fig.show()

Update crossing and plot again

In [ ]:
collider.vars["on_x1"] = 160.000
collider.vars["on_sep1"] = 0.0
collider.vars["phi_IR1"] = 90.000

collider.vars["on_x2h"] = 0.000
collider.vars["on_sep2h"] = -0.01  # 1.000
collider.vars["on_x2v"] = 200.000
collider.vars["on_sep2v"] = 0.000
collider.vars["phi_IR2"] = 90.000

collider.vars["on_x5"] = 160.000
collider.vars["on_sep5"] = 0.0
collider.vars["phi_IR5"] = 0.000

collider.vars["on_x8h"] = 0.000
collider.vars["on_sep8h"] = -0.01  # -1.000
collider.vars["on_x8v"] = 200.000
collider.vars["on_sep8v"] = 0.000
collider.vars["phi_IR8"] = 180.000

In [ ]:
fig = plot_beta()
fig.show()

In [ ]:
fig = plot_orbit()
fig.show()